# Implementation: Normalized Cuts and Image Segmentation

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)  
This work by Wei-Chen Pan is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

## Reference
This work is based on the theory and algorithms provided in [1].

[[1]](https://www.sciencedirect.com/science/article/pii/S089812210500204X) J. Shi and J. Malik.  Normalized Cuts and Image Segmentation.  *IEEE Transactions on Pattern Analysis and Machine Intelligence*, 22:888&ndash;905, 2000.

## Pseudocode

#### Input
1. $G=(V,E)$, taking each pixel as a node $(V)$ and connecting each pair of pixels in radius$(r)$ by an edge $(E)$.
2. $w_{ij}$, the weight on the edge connecting two nodes to be a measure of the similarity between the two nodes

#### Output
&#8195; &#8195;&#8194; $x_i = 1$ if node $i$ is in partition $A$ and $0$, otherwise.

#### Steps
1. Let $D$ be the $N \times N$ diagonal matrix whose $i,i$-entry is $\sum_{j} w_{ij}$
2. Let $W$ be the $N \times N$ symmetrical matrix whose $i,j$-entry is $w_{ij}$
3. Solve the eigenproblem $(D-W) 𝑦=λD𝑦$ such that the eigenvalues are in descending. 
4. Split y by some splitting point, and also ignore all those eigenvectors which have smoothly varying eigenvector values.
(threshold : 0.06 ratio between the minimum and maximum values of the eigenvector values)
5. $𝑦=(1+ x)-b(1- x)$, where $b=\frac{\sum_{x_i>0}}{\sum_{x_i<0}} ⇒ x=\frac{y−1+b}{1+b}$

## Implementation

In [1]:
import scipy.linalg as LA
import numpy as np
import pandas as pd
from collections import Counter
np.set_printoptions(precision=2)

##### Input data
data_s is an $n \times 2$ matrix that represents the 2-dim coordinate (spatial location) of $n$ nodes.  
data_F is the feature vectors.  
r is the boundary that if the distance between node i and j less than r, than there exist weight on the edge between them.

In [2]:
class Ncut:
    def __init__(self, data_s, data_F, r, c=10):
        self.data_s = data_s
        self.data_F = data_F
        self.n, self.m = data_s.shape
        self.r = r
        self.W = np.zeros(self.n**2).reshape(self.n,self.n)
        self.D = np.zeros(self.n**2).reshape(self.n,self.n)
        self.L = []
        self.x = []
        self.c = c           #The number of cuts when ignoring the ratio in ig_partition.
        self.ig_x = []
    
    def lap_matrix(self):
        for i in range(self.n):
            for j in range(self.n):
                dsij = np.linalg.norm(np.subtract(self.data_s[i],self.data_s[j]))
                if dsij < self.r and i != j:
                    dFij = np.linalg.norm(self.data_F[i]-self.data_F[j])
                    self.W[i,j] = np.exp(-dFij**2) * np.exp(-dsij**2)
                else:
                    self.W[i,j] = 0
        
        self.D = np.zeros_like(self.W)    # degrees matrix
        self.D[np.arange(self.n), np.arange(self.n)] = self.W.sum(axis=1)
#         for i in range(self.n):
#             self.D[i,i] = np.sum(self.W[i])
        self.L = self.D - self.W
#         self.L = np.subtract(self.D,self.W)
        return self.L
    
    def full_partition(self):
        eigs, vecs = LA.eigh(self.L, self.D)
        y = vecs #* np.array([-1,1,1,-1,-1,-1,-1,1,-1])
        split_y = np.zeros((self.n,self.n))
#         split_y = np.zeros(self.n**2).reshape(self.n,self.n)
        for j in range(self.n):     #split
            mu = y[:,j].mean()
            for i in range(self.n):
                if y[i,j] >= mu:
                    split_y[i,j] = 1
                else:
                    split_y[i,j] = 0
        self.x = split_y
        return self.x
    
    def ig_partition(self):
        #ignore those eigenvectors that the ratio > 0.06.
        eigs, vecs = LA.eigh(self.L, self.D)
        y = vecs #* np.array([-1,1,1,-1,-1,-1,-1,1,-1])
        u = np.zeros(self.n)
        for j in range(self.n):
            z = np.array(Counter(pd.cut(y[:,j],self.c)).most_common())[:,1]
            if z.shape[0]==self.c: u[j] = min(z)/max(z)   #u is the ratio between min and max of the eigenvector values.
            else: u[j] = 0
        ig_y = np.zeros_like(y).astype(float)
        split_y = np.zeros_like(y).astype(float)
        for i in range(self.n):
            if u[i]<0.06: ig_y[:,i] = y[:,i]   #ignore
            else: ig_y[:,i] = 0
        for j in range(self.n):                #split
            for i in range(self.n):
                if ig_y[i,j] > np.mean(ig_y[:,j]):
                    split_y[i,j] = 1
                else:
                    split_y[i,j] = 0
        self.ig_x = split_y
        return self.ig_x

## Examples

In [3]:
s = np.array([[1,1],[1,2],[1,3],[2,1],[2,2],[2,3],[3,1],[3,2],[3,3]])   #spacial
# F = np.array([1,2,3,1,4,3,1,2,6])                                       #feature (can be vector)
F = np.array([1,2,3,4,5,6,7,8,9])                                       #feature (can be vector)

model = Ncut(s, F, r=2, c=3)

In [4]:
model.lap_matrix()

array([[ 1.35e-01, -1.35e-01,  0.00e+00, -4.54e-05, -1.52e-08,  0.00e+00,
         0.00e+00,  0.00e+00,  0.00e+00],
       [-1.35e-01,  2.73e-01, -1.35e-01, -2.48e-03, -4.54e-05, -1.52e-08,
         0.00e+00,  0.00e+00,  0.00e+00],
       [ 0.00e+00, -1.35e-01,  1.38e-01,  0.00e+00, -2.48e-03, -4.54e-05,
         0.00e+00,  0.00e+00,  0.00e+00],
       [-4.54e-05, -2.48e-03,  0.00e+00,  1.38e-01, -1.35e-01,  0.00e+00,
        -4.54e-05, -1.52e-08,  0.00e+00],
       [-1.52e-08, -4.54e-05, -2.48e-03, -1.35e-01,  2.76e-01, -1.35e-01,
        -2.48e-03, -4.54e-05, -1.52e-08],
       [ 0.00e+00, -1.52e-08, -4.54e-05,  0.00e+00, -1.35e-01,  1.38e-01,
         0.00e+00, -2.48e-03, -4.54e-05],
       [ 0.00e+00,  0.00e+00,  0.00e+00, -4.54e-05, -2.48e-03,  0.00e+00,
         1.38e-01, -1.35e-01,  0.00e+00],
       [ 0.00e+00,  0.00e+00,  0.00e+00, -1.52e-08, -4.54e-05, -2.48e-03,
        -1.35e-01,  2.73e-01, -1.35e-01],
       [ 0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00, -1.52e-08, -4.54e-05

###### For the following matrix. Each column is a cut(except for the first column). The index =1 means those nodes are a partition.
The values in first column are not all 1 is because of the numerical error for computing eigenvectors. Shown in the upper cell.

In [5]:
model.full_partition()

array([[1., 0., 1., 0., 1., 1., 0., 1., 1.],
       [1., 0., 1., 1., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 1., 1.],
       [1., 0., 0., 0., 1., 0., 1., 1., 1.],
       [1., 0., 0., 1., 0., 1., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 0., 0., 0., 1.]])

In [6]:
model.ig_partition()                #because the values(ratio) in u are all <0.06, so the ignored partitions is same as the full one.

array([[0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1.]])